In [1]:
from gurobipy import *
import itertools

This is the optimization for a rack by rack basis in the system. 

In [2]:
m = Model();

Academic license - for non-commercial use only


The decision varible is $x_{ijr}$ which is the number of chips where $i$ is the specific chip type, the $j$ is the orientation, and $r$ is the rack where the chip is assigned.

In [3]:
chips, powerdraw, quantity = multidict({
    '1080':[150,10],
    '2080':[180,10]
})

In [4]:
racks , slots = multidict({
    'alpha':48,
    'beta':48
})

The orientations are as follows: 
$$j_1\rightarrow(L1,L2)$$
$$j_2\rightarrow(L2,L3)$$
$$j_3\rightarrow(L1,L3)$$
$$s.t.$$
$$\theta = \{j_1,j_2,j_3\}$$

In [5]:
i = chips

In [6]:
orientations = 3
j = range(orientations)

In [7]:
r = racks

In [8]:
index = itertools.product(i,j,r)

In [9]:
x = m.addVars(index)

In [10]:
# make sure all the chips have somewhere to go
m.addConstrs(x.sum(chip,'*','*') == quantity[chip] for chip in chips)

{'1080': <gurobi.Constr *Awaiting Model Update*>,
 '2080': <gurobi.Constr *Awaiting Model Update*>}

In [20]:
L = m.addVars(racks,range(3))

In [21]:
L

{('alpha', 0): <gurobi.Var *Awaiting Model Update*>,
 ('alpha', 1): <gurobi.Var *Awaiting Model Update*>,
 ('alpha', 2): <gurobi.Var *Awaiting Model Update*>,
 ('beta', 0): <gurobi.Var *Awaiting Model Update*>,
 ('beta', 1): <gurobi.Var *Awaiting Model Update*>,
 ('beta', 2): <gurobi.Var *Awaiting Model Update*>}

In [17]:
# we want to perfer l1-l2, l2-l3, l1-l3 in that order
m.addConstrs(L[rack,0]>=L[rack,1] for rack in racks)
m.addConstrs(L[rack,1]>=L[rack,2] for rack in racks)

KeyError: ('alpha', 0)

In [12]:
w = m.addVars(r)

In [14]:
# calculate the load on each phase for each rack
# for rack in racks
#  for phase in 

KeyError: (range(0, 2), 1)